In [107]:
import pandas as pd
from datetime import datetime
import openpyxl
import mysql.connector.connection


CONFIGURACION

In [108]:
month = 3
year = 2025

                                                            --- NO TOCAR ---                                                                    
                                                            

In [109]:
# Crear un objeto datetime con el mes y año (el día puede ser arbitrario, por ejemplo, 1)
date_obj = datetime(year, month, 1)  
date_str = date_obj.strftime("%m/%Y")

name_month = date_obj.strftime("%B")
file_name = f"{name_month} {year}.xlsx"
file_name

'March 2025.xlsx'

In [110]:
def conexion_db(host, user, password, database):
    try:
        cnx = mysql.connector.connect(host=host, user=user, password=password, database=database)
        print("✅ Conexion a base de datos establecida correctamente")
        return cnx
    except Exception as error:
        print(f"❌ Ocurrio un error al intentarse conectar a la base de datos\
              Error: {error}")
def query(cnx, query):
    try:
        cursor = cnx.cursor()
        cursor.execute(query)
        resultado = cursor.fetchall()
        if resultado is not None:
            print("✅ Se extrajeron los datos correctamente")
        else:
            print("❌ La consulta vino vacia")
        cursor.close()
        cnx.close()
        print("✅ Se cerro la conexion con base de datos")
        return resultado
    except Exception as error_query:
        print(f"❌ Ocurrio un error al intentar ejecutar la consulta\
              Error: {error_query}")


In [ ]:
### Consutla extraccion de datos
# PAYROLL
payroll_extraction_query = f"""
SELECT
legajo,
documento, 
apellidos, 
nombres, 
equipo, 
fecha, 
codigo, 
descripcion, 
horas_programadas, 
horas_trabajadas,
usuario_neotel,
usuario_avaya,
campana, 
sub_campana, 
area, 
puesto
FROM payroll
WHERE MONTH(fecha) = {month} AND YEAR(fecha) = {year};""" 

columnas_payroll = ["legajo","documento", "apellidos", "nombres","equipo","fecha", "codigo", "descripcion", 
"horas_programadas","horas_trabajadas","usuario_neotel","usuario_avaya","campana", "sub_campana","area", "puesto"]

# NOMINA
nomina_extraction_query = """
SELECT legajo, documento, apellidos, nombres FROM asesores;
"""
columnas_nomina = ["legajo","documento", "apellidos", "nombres"]

# EXTRA
lista_columnas_vacias = ["Hs Extras", "Dep. Judicial", "Prepaga","Enlazados", "Ant. de Sueldos", "Adic. Ventas", "OBSERVACIONES"]


In [112]:
# Consumimos payroll
cnx = conexion_db("172.16.20.15", "mariano_orozco", "goodTimes194.kilp", "db_omnia")
resultado = query(cnx,payroll_extraction_query)
payroll = pd.DataFrame(resultado, columns=columnas_payroll)
payroll.head()

✅ Conexion a base de datos establecida correctamente
✅ Se extrajeron los datos correctamente
✅ Se cerro la conexion con base de datos


,legajo,documento,apellidos,nombres,equipo,fecha,codigo,descripcion,horas_programadas,horas_trabajadas,usuario_neotel,usuario_avaya,campana,sub_campana,area,puesto
0,1098,36788122,MARTINEZ,MARIA FLORENCIA,"NUNES CARRERO, MARIANA CAROLINA",2025-03-03,DNL,Día No Laborable,8.0,0.0,NaN,NaN,None,None,None,None
1,1098,36788122,MARTINEZ,MARIA FLORENCIA,"NUNES CARRERO, MARIANA CAROLINA",2025-03-04,DNL,Día No Laborable,8.0,0.0,NaN,NaN,None,None,None,None
2,1098,36788122,MARTINEZ,MARIA FLORENCIA,"NUNES CARRERO, MARIANA CAROLINA",2025-03-05,V,Vacaciones,0.0,0.0,NaN,NaN,None,None,None,None
3,1098,36788122,MARTINEZ,MARIA FLORENCIA,"NUNES CARRERO, MARIANA CAROLINA",2025-03-06,V,Vacaciones,0.0,0.0,NaN,NaN,None,None,None,None
4,1098,36788122,MARTINEZ,MARIA FLORENCIA,"NUNES CARRERO, MARIANA CAROLINA",2025-03-07,V,Vacaciones,0.0,0.0,NaN,NaN,None,None,None,None


In [113]:
cnx = conexion_db("172.16.20.15", "mariano_orozco", "goodTimes194.kilp", "db_omnia")
resultado = query(cnx,nomina_extraction_query)
nomina = pd.DataFrame(resultado, columns=columnas_nomina)
nomina.head()

✅ Conexion a base de datos establecida correctamente
✅ Se extrajeron los datos correctamente
✅ Se cerro la conexion con base de datos


,legajo,documento,apellidos,nombres
0,1317,18902640,CACERES PEREIRA,ANDRES FEDERICO
1,1600,19113210,RESTREPO ORDOSGOITIA,ARIANA ESTEFANNY
2,1601,19113447,RODRIGUEZ ALFONZO,NAIYELY HECNAY
3,884,19116652,APABLAZA GEREZ,ELIZABETH SABRINA
4,1229,23081765,PEREYRA,MARIA NORMA


In [ ]:
### Creacion de columnas en nomina que me realicen el conteo de dias dependiendo el n° de legajo
nomina["Basico"] = None

# Contar cuántas veces aparece cada legajo con código "x" (D. Enfermedad)
enfermedad = payroll[payroll['codigo'] == 'CM'].groupby('legajo').size().reset_index(name='D. Enfermedad')

# Contar cuántas veces aparece cada legajo con código "v" (Vacaciones)
vacaciones = payroll[payroll['codigo'] == 'V'].groupby('legajo').size().reset_index(name='Vacaciones')

# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
feriado = payroll[payroll['codigo'] == 'DNL'].groupby('legajo').size().reset_index(name='Feriado')

# Contar cuántas veces aparece cada legajo con código "L" (Ausencias Injustificadas)
licencias = payroll[payroll['codigo'].isin(['LESP', 'LMED', 'MTM', 'LPS', 'CMh', 'LM', 'ART', 'LFF'])] \
          .groupby('legajo').size().reset_index(name='Licencia')

# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
mudanza = payroll[payroll['codigo'] == 'M'].groupby('legajo').size().reset_index(name='D. Mudanza')


# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
estudio = payroll[payroll['codigo'] == 'CE'].groupby('legajo').size().reset_index(name='D. Estudio')


# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
suspension = payroll[payroll['codigo'] == 'S'].groupby('legajo').size().reset_index(name='D. Suspension')


# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
tramite = payroll[payroll['codigo'] == 'DT'].groupby('legajo').size().reset_index(name='D. Tramite')


# Contar cuántas veces aparece cada legajo con código "a" (Ausencias Injustificadas)
falta_injustificada = payroll[payroll['codigo'] == 'FI'].groupby('legajo').size().reset_index(name='Falta Inj.')



# Combinar todos los conteos en un solo DataFrame
conteos = enfermedad.merge(vacaciones, on='legajo', how='outer') \
                    .merge(feriado, on='legajo', how='outer') \
                    .merge(licencias, on='legajo', how='outer') \
                    .merge(mudanza, on='legajo', how='outer') \
                    .merge(estudio, on='legajo', how='outer') \
                    .merge(suspension, on='legajo', how='outer') \
                    .merge(tramite, on='legajo', how='outer') \
                    .merge(falta_injustificada, on='legajo', how='outer') 

nomina = nomina.merge(conteos, on='legajo', how='left')


for col in lista_columnas_vacias:
    nomina[col] = None


nomina.head()

,legajo,documento,apellidos,nombres,Basico,D. Enfermedad,Vacaciones,Feriado,Licencia,D. Mudanza,...,D. Suspension,D. Tramite,Falta Inj.,Hs Extras,Dep. Judicial,Prepaga,Enlazados,Ant. de Sueldos,Adic. Ventas,OBSERVACIONES
0,1317,18902640,CACERES PEREIRA,ANDRES FEDERICO,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,None,None
1,1600,19113210,RESTREPO ORDOSGOITIA,ARIANA ESTEFANNY,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,None,None
2,1601,19113447,RODRIGUEZ ALFONZO,NAIYELY HECNAY,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,None,None
3,884,19116652,APABLAZA GEREZ,ELIZABETH SABRINA,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,None,None
4,1229,23081765,PEREYRA,MARIA NORMA,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,None,None


In [115]:
# exportamos a excel para analizar 
try:
    nomina = nomina.sort_values(by=['apellidos', 'nombres'], ascending=[True, True])
    nomina.to_excel(f"{file_name}", index=False)
    print(f"✅ Se extrajo correctamten el archivo {file_name}")
except Exception as error_exportacion:
    print(f"❌ Ocurrio un error al exportar el archivo\n \
          Error: {error_exportacion}")

✅ Se extrajo correctamten el archivo March 2025.xlsx
